# Almacenamiento

Muchas industrias han puesto un gran énfasis en las tecnologías de almacenamiento de datos en la nube para ayudar a facilitar la creciente demanda de datos. Hay muchas opciones disponibles para el almacenamiento de datos que se adaptan a sus necesidades, con una escala aparentemente infinita. Incluso con muchas nuevas opciones de almacenamiento disponibles en la nube, Amazon S3 sigue siendo un bloque de construcción potente y fundamental para muchos casos de uso. Es increíble pensar que se lanzó hace más de 15 años. Con el tiempo, se han añadido muchas características y se han lanzado nuevos servicios de almacenamiento. Existen múltiples opciones de almacenamiento para satisfacer los requisitos de seguridad (por ejemplo, el cifrado del servicio de gestión de claves [**KMS**]) y reducir los costes (por ejemplo, *S3 Intelligent-Tiering*). Garantizar la seguridad y la disponibilidad de los datos es un reto al que se enfrentan todos los desarrolladores y arquitectos.

Los servicios de almacenamiento disponibles en AWS permiten la integración con otros servicios de AWS para proporcionar formas a los desarrolladores y arquitectos de aplicaciones que se integran con muchos servicios de AWS. Estos servicios también pueden utilizarse para sustituir los sistemas de almacenamiento heredados que se ejecutan y operan con entornos on-premise. Por ejemplo:

* S3 puede utilizarse para invocar automáticamente funciones Lambda en operaciones de objetos como la carga.

* EFS puede utilizarse con EC2 para sustituir los sistemas de archivos compartidos existentes proporcionados por los servidores del sistema de archivos de red (NFS).

* FSx para Windows puede utilizarse para sustituir a los servidores de archivos basados en Windows para sus cargas de trabajo de EC2.

* EBS sustituye a los objetivos de Canal de Fibra y de Interfaz de Sistemas de Computadoras Pequeñas de Internet (iSCSI) proporcionando dispositivos de bloque, y ofrece muchas opciones de rendimiento para satisfacer los requisitos de rendimiento.

### Uso de las políticas del ciclo de vida de S3 para reducir los costes de almacenamiento

#### Problema
Necesita trasladar los objetos a los que se accede con poca frecuencia a un nivel de almacenamiento más rentable sin que ello afecte al rendimiento o añada una sobrecarga operativa.

#### Solución
Cree una regla del ciclo de vida de S3 para realizar la transición de los objetos a la clase de almacenamiento de acceso infrecuente (IA) de S3 después de un período de tiempo predefinido de 30 días. A continuación, aplique esta política de ciclo de vida a su cubo de S3.

<img src="https://docs.aws.amazon.com/AmazonS3/latest/userguide/images/lifecycle-transitions-v3.png" width="900">


In [1]:
import boto3
import json
import random
from pprint import pprint 

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [2]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

Bucket name: cookbook-86954


Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) están desactivadas por defecto.

In [4]:
response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

#### Cree un ciclo de vida que aplicará a su bucket de S3:

Cuando añade una configuración de S3 Lifecycle a un bucket, por lo general se produce un cierto desfase antes de que una configuración de ciclo de vida nueva o actualizada se propague totalmente a todos los sistemas de Amazon S3. Habrá un retraso de algunos minutos antes de que la configuración entre en vigor completamente. Este retraso también se puede producir cuando elimina una configuración de S3 Lifecycle.

**NOTA:** Cada bucket tiene 1 configuración de ciclo de vida, que puede tener hasta 1000 reglas, *put_bucket_lifecycle_configuration* crea una nueva configuración de ciclo de vida para el cubo o **sustituye** una configuración de ciclo de vida existente. Si quieres actualizar la configuración del ciclo de vida, tienes que usar *get_bucket_lifecycle_configuration* para recuperar las reglas existentes, modificarlas y luego usar *put_bucket_lifecycle_configuration* para sobrescribir la configuración existente.

In [129]:
# Create a json data:
json_data = {
    "Rules": [
        {
            "ID": "Move all objects to Standard Infrequently Access",
            "Filter": {
                "Prefix": ""
            },
            "Status": "Enabled",
            "Transitions": [
                {
                    "Days": 30,                   
                    "StorageClass": "STANDARD_IA"
                }
            ]
        }
    ]
}

In [130]:
# Apply the Lifecycle rule configuration:
response = s3_client.put_bucket_lifecycle_configuration(
    Bucket=bucket_name,
    LifecycleConfiguration=json_data
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Lifecycle configuration applied.")
else:
    print("ERROR: Something went wrong.")

Lifecycle configuration applied.


* Una regla de ciclo de vida ayuda a automatizar la transición a una clase de almacenamiento diferente para algunos o todos los objetos de un cubo (los prefijos, las etiquetas y los nombres de los objetos pueden utilizarse como filtros para las reglas de ciclo de vida). Para obtener una lista completa de las capacidades de las reglas del ciclo de vida, consulte la documentación.

* Cuando añade una configuración de ciclo de vida a un bucket, las reglas de configuración se aplican a los objetos existentes y a los objetos que añade posteriormente. Por ejemplo, si hoy añade una regla de configuración de Lifecycle con una acción de vencimiento que causa que objetos con un prefijo específico expiren 30 días después de la creación, Amazon S3 pondrá en cola de eliminación cualquier objeto existente con más de 30 días de antigüedad.

#### Controles de validación

In [101]:
# Get the Lifecycle configuration for your bucket:
response = s3_client.get_bucket_lifecycle_configuration(
    Bucket=bucket_name
)

print(json.dumps(response['Rules'], indent=4))

[
    {
        "ID": "Move all objects to Standard Infrequently Access",
        "Prefix": "",
        "Status": "Enabled",
        "Transitions": [
            {
                "Days": 30,
                "StorageClass": "STANDARD_IA"
            }
        ]
    }
]


In [40]:
# Upload a object to S3:
response = s3.Object(bucket_name, 'object_to_delte.json').put(Body=open('ip-ranges.json', 'rb'))

In [44]:
# Check the storage class for the object:
response = s3_client.list_objects(
    Bucket=bucket_name,
)

In [59]:
pprint(response['Contents'][0])

{'ETag': '"a4c4f220536a3a2bbfb53e6772039d1d"',
 'Key': 'object_to_delte.json',
 'LastModified': datetime.datetime(2022, 10, 15, 14, 30, 10, tzinfo=tzutc()),
 'Owner': {'DisplayName': 'lab+LabServices-Prod-5828',
           'ID': '458f686c07dc2a102045f8b9923f246690c2ba35949449f75b715f1020bd5116'},
 'Size': 1348061,
 'StorageClass': 'STANDARD'}


Después de 30 días verá que la clase de almacenamiento del objeto es STANDARD_IA tras ejecutar el mismo comando.

**TIP:** "Días" en la acción de transición debe ser mayor o igual a 30 para StorageClass STANDARD_IA. Otras clases de almacenamiento permiten tiempos de transición más cortos para satisfacer sus necesidades. Para obtener una lista de todas las clases de almacenamiento disponibles con tiempos de transición para las reglas del ciclo de vida, consulte el [documento de soporte](https://oreil.ly/6jPLh).

#### Discusión
Cuando se cargan objetos en un bucket de S3, si no se especifica la clase de almacenamiento, se utiliza la clase de almacenamiento estándar por defecto. Amazon S3 dispone de varias clases de almacenamiento que pueden ser más rentables para el almacenamiento a largo plazo, al tiempo que se adaptan a sus requisitos de rendimiento y resistencia. Si no puede cambiar su aplicación para especificar los niveles de almacenamiento para las cargas de objetos, las reglas del ciclo de vida pueden ayudar a automatizar la transición a la clase de almacenamiento que desee. Las reglas del ciclo de vida pueden aplicarse a algunos o a todos los objetos dentro de un cubo con un filtro.

Como su nombre indica, S3 Infrequent Access es una clase de almacenamiento que proporciona un coste reducido (en comparación con la clase de almacenamiento S3 Standard) para los datos almacenados para los objetos a los que rara vez se accede. Esto proporciona el mismo nivel de redundancia para sus datos dentro de una Región por un coste reducido, pero el coste asociado al acceso a los objetos es ligeramente superior. Si sus patrones de acceso a los datos son impredecibles, y todavía desea optimizar su almacenamiento en S3 para el costo, el rendimiento y la capacidad de recuperación, eche un vistazo a S3 Intelligent-Tiering en la siguiente receta.

#### Reto 1
Configure la regla del ciclo de vida para que se aplique sólo a los objetos basados en etiquetas de nivel de objeto.

In [137]:
# Get the Lifecycle configuration for your bucket:
response = s3_client.get_bucket_lifecycle_configuration(
    Bucket=bucket_name
)
actual_rules = response['Rules']
print("Cantidad de reglas actuales: {}".format(len(actual_rules)))

Cantidad de reglas actuales: 2


Agregar la politica actual para no sobrescribirla

In [134]:
json_data = {
    'Rules': [
        {
            'ID': 'Transition tagged objects to S3 Glacier Deep Archive after 3 years',
            'Filter': {
                    'Tag': {
                            'Key': 'TransitionDeepArchive',
                            'Value': '1095'
                        }
            }, 
            'Status': 'Enabled', 
            'Transitions': [
                {
                'Days': 1095,
                'StorageClass': 'DEEP_ARCHIVE'
                }
            ]
        },
    actual_rules[0]  ## Add actual policy
    ],
}

In [135]:
# Apply the Lifecycle rule configuration:
response = s3_client.put_bucket_lifecycle_configuration(
    Bucket=bucket_name,
    LifecycleConfiguration=json_data
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Lifecycle configuration applied.")
else:
    print("ERROR: Something went wrong.")

Lifecycle configuration applied.


In [136]:
# Upload a object to S3
response = s3.Object(bucket_name, 'new-report.png').put(
    Body=open('new-report.png', 'rb'),
    Tagging='TransitionDeepArchive=1095'
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Object uploaded.")

Object uploaded.


Reglas: STANDARD_IA , ONEZONE_IA , INTELLIGENT_TIERING , GLACIER_IR , GLACIER , DEEP_ARCHIVE